## data preprocessing

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ast
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [2]:
#  Load the data
domain1_train_data = pd.read_json('domain1_train_data.json', lines=True)
domain2_train_data = pd.read_json('domain2_train_data.json', lines=True)


print(domain1_train_data.head())
print(domain2_train_data.head())


                                                text  label  id
0  [16, 231, 543, 5, 15, 43, 8282, 94, 231, 1129,...      1   0
1  [16, 4046, 138, 10, 2, 1809, 2007, 3763, 14, 4...      1   1
2  [1108, 16550, 3, 6168, 3, 160, 284, 19, 49, 46...      1   2
3  [1802, 27, 16, 25, 48, 451, 632, 3, 2, 2164, 2...      1   3
4  [16, 19, 302, 93, 97, 43, 952, 118, 1, 16, 528...      1   4
                                                text  label    id
0  [12, 920, 7, 1266, 28, 9884, 1640, 116, 11, 13...      1  5000
1  [783, 397, 253, 5797, 9379, 22, 793, 11838, 10...      1  5001
2  [888, 14851, 323, 9, 27, 1377, 584, 195, 3, 13...      1  5002
3  [228, 1161, 5815, 379, 9, 941, 10, 2, 316, 4, ...      1  5003
4  [736, 19, 37, 813, 45, 6723, 27, 626, 8, 2, 34...      1  5004


In [3]:
# get machine and human data
machine = domain2_train_data[domain2_train_data['label'] == 0]
human = domain2_train_data[domain2_train_data['label'] == 1]

# count the number of samples in each class
n_machine = len(machine)
n_human = len(human)

# if the number of samples in 'machine' is greater than the number of samples in 'human'
if n_machine > n_human:
    machine = machine.sample(n_human)

# combine the balanced data
domain2_train_data_balanced = pd.concat([machine, human])

print(domain2_train_data_balanced.head())

                                                    text  label     id
3833   [12, 155, 7, 193, 63, 10, 6467, 857, 14, 2672,...      0   8833
11415  [2522, 988, 931, 3, 52, 206, 7143, 988, 931, 3...      0  16415
6312   [8, 2, 446, 4, 3366, 619, 156, 6463, 7692, 3, ...      0  11312
6296   [43, 7598, 865, 8, 7708, 172, 9, 81, 218, 5, 4...      0  11296
3667   [7760, 323, 87, 14, 8049, 13, 22, 2203, 18, 98...      0   8667


In [4]:
# combine the two datasets
combined_data = pd.concat([domain1_train_data, domain2_train_data_balanced])


# get the features and labels
X = combined_data['text']
y = combined_data['label']


# # split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:
# 将词语列表转换为字符串
X_train_str = [' '.join(map(str, lst)) for lst in X_train]
X_val_str = [' '.join(map(str, lst)) for lst in X_val]

# tf-idf向量化
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train_str)
X_val_vec = vectorizer.transform(X_val_str)

## adaboost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Define the base estimator (weak learner)
base_estimator = DecisionTreeClassifier(max_depth=1)

# Define the AdaBoost classifier
adaboost_model = AdaBoostClassifier(base_estimator=base_estimator, random_state=42)

# Define the hyperparameter grid for grid search
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.1, 0.5, 1.0]
}

# Perform grid search with cross-validation 
grid_search = GridSearchCV(estimator=adaboost_model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1)
grid_search_result = grid_search.fit(X_train_vec, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search_result.best_params_)

# Evaluate the best model on the validation set
best_adaboost_model = grid_search_result.best_estimator_
adaboost_predictions = best_adaboost_model.predict(X_val_vec)
accuracy = accuracy_score(y_val, adaboost_predictions)
print("Validation Accuracy:", accuracy)


Fitting 3 folds for each of 9 candidates, totalling 27 fits
Best Hyperparameters: {'learning_rate': 0.5, 'n_estimators': 200}
Validation Accuracy: 0.68828125


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Define the base estimator (weak learner)
base_estimator = DecisionTreeClassifier(max_depth=1)

# Define the AdaBoost classifier
adaboost_model = AdaBoostClassifier(base_estimator=base_estimator, random_state=42)

# Define the hyperparameter grid for grid search
param_grid = {
    'n_estimators': [200, 250, 300, 350],
    'learning_rate': [0.5]
}

# Perform grid search with cross-validation 
grid_search = GridSearchCV(estimator=adaboost_model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1)
grid_search_result = grid_search.fit(X_train_vec, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search_result.best_params_)

# Evaluate the best model on the validation set
best_adaboost_model_2 = grid_search_result.best_estimator_
adaboost_predictions_2 = best_adaboost_model_2.predict(X_val_vec)
accuracy_2 = accuracy_score(y_val, adaboost_predictions_2)
print("Validation Accuracy:", accuracy_2)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
Best Hyperparameters: {'learning_rate': 0.5, 'n_estimators': 300}
Validation Accuracy: 0.6875

replicate Best Hyperparameters: {'learning_rate': 0.5, 'n_estimators': 200}

In [8]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report

# Initialize the AdaBoost classifier with the best hyperparameters
best_adaboost_model = AdaBoostClassifier(learning_rate=0.5, n_estimators=200, random_state=42)

# Fit the model on the augmented training data
best_adaboost_model.fit(X_train_vec, y_train)

# Make predictions on the validation set
adaboost_predictions = best_adaboost_model.predict(X_val_vec)

# Generate the classification report
classification_rep = classification_report(y_val, adaboost_predictions)

# Print the classification report
print("Classification Report for AdaBoost:")
print(classification_rep)

Classification Report for AdaBoost:
              precision    recall  f1-score   support

           0       0.68      0.66      0.67       778
           1       0.69      0.71      0.70       822

    accuracy                           0.68      1600
   macro avg       0.68      0.68      0.68      1600
weighted avg       0.68      0.68      0.68      1600



In [7]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report

# Initialize the AdaBoost classifier with the best hyperparameters
best_adaboost_model = AdaBoostClassifier(learning_rate=0.5, n_estimators=200, random_state=42)

# Fit the model on the augmented training data
best_adaboost_model.fit(X_val_vec, y_val)

# Make predictions on the validation set
adaboost_predictions = best_adaboost_model.predict(X_val_vec)

# Generate the classification report
classification_rep = classification_report(y_val, adaboost_predictions)

# Print the classification report
print("Classification Report for AdaBoost:")
print(classification_rep)


Classification Report for AdaBoost:
              precision    recall  f1-score   support

           0       0.93      0.90      0.91       778
           1       0.91      0.94      0.92       822

    accuracy                           0.92      1600
   macro avg       0.92      0.92      0.92      1600
weighted avg       0.92      0.92      0.92      1600



## adaboost with svm

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

# Convert the lists of integers into strings
X_train_str = X_train.apply(lambda x: ' '.join(map(str, x)))

# Initialize the TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the training data
X_train_vec = vectorizer.fit_transform(X_train_str)

# Define the base estimator (SVM)
base_estimator = SVC(probability=True, random_state=42)

# Initialize the AdaBoost classifier with SVM as the base estimator
adaboost_svm_model = AdaBoostClassifier(base_estimator=base_estimator, random_state=42)

# Define hyperparameters for grid search
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.1, 0.5, 1.0]
}

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(estimator=adaboost_svm_model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1)
grid_search.fit(X_train_vec, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Get the best AdaBoost model with SVM as the base estimator
best_adaboost_svm_model = grid_search.best_estimator_

# Assuming you have validation data (X_val, y_val)
# Convert the validation data using the same vectorizer
X_val_str = X_val.apply(lambda x: ' '.join(map(str, x)))
X_val_vec = vectorizer.transform(X_val_str)

# Make predictions on the validation set
adaboost_svm_predictions = best_adaboost_svm_model.predict(X_val_vec)

# Calculate accuracy
accuracy = accuracy_score(y_val, adaboost_svm_predictions)
print("Validation Accuracy:", accuracy)


Logistic Regression Performance:
              precision    recall  f1-score   support

           0       0.74      0.74      0.74       653
           1       0.73      0.73      0.73       627

    accuracy                           0.73      1280
   macro avg       0.73      0.73      0.73      1280
weighted avg       0.73      0.73      0.73      1280







                                                text  label  id
0  [16, 231, 543, 5, 15, 43, 8282, 94, 231, 1129,...      1   0
1  [16, 4046, 138, 10, 2, 1809, 2007, 3763, 14, 4...      1   1
2  [1108, 16550, 3, 6168, 3, 160, 284, 19, 49, 46...      1   2
3  [1802, 27, 16, 25, 48, 451, 632, 3, 2, 2164, 2...      1   3
4  [16, 19, 302, 93, 97, 43, 952, 118, 1, 16, 528...      1   4
                                                text  label    id
0  [12, 920, 7, 1266, 28, 9884, 1640, 116, 11, 13...      1  5000
1  [783, 397, 253, 5797, 9379, 22, 793, 11838, 10...      1  5001
2  [888, 14851, 323, 9, 27, 1377, 584, 195, 3, 13...      1  5002
3  [228, 1161, 5815, 379, 9, 941, 10, 2, 316, 4, ...      1  5003
4  [736, 19, 37, 813, 45, 6723, 27, 626, 8, 2, 34...      1  5004
text	label	id
0	[16, 231, 543, 5, 15, 43, 8282, 94, 231, 1129,...	1	0
1	[16, 4046, 138, 10, 2, 1809, 2007, 3763, 14, 4...	1	1
2	[1108, 16550, 3, 6168, 3, 160, 284, 19, 49, 46...	1	2
3	[1802, 27, 16, 25, 48, 451, 632, 3, 2, 2164, 2...	1	3
4	[16, 19, 302, 93, 97, 43, 952, 118, 1, 16, 528...	1	4
...	...	...	...
4995	[43, 529, 16, 19, 775, 201, 20, 48, 10, 550, 2...	0	4995
4996	[12, 158, 97, 5, 543, 174, 1396, 2, 506, 287, ...	0	4996
4997	[15319, 27775, 9, 27, 2847, 7207, 8, 3234, 1, ...	0	4997
4998	[16, 373, 177, 76, 5, 35, 1342, 1318, 196, 16,...	0	4998
4999	[15, 25, 12264, 20611, 29, 25, 2, 1673, 900, 2...	0	4999
5000 rows × 3 columns

text	label	id
0	[12, 920, 7, 1266, 28, 9884, 1640, 116, 11, 13...	1	5000
1	[783, 397, 253, 5797, 9379, 22, 793, 11838, 10...	1	5001
2	[888, 14851, 323, 9, 27, 1377, 584, 195, 3, 13...	1	5002
3	[228, 1161, 5815, 379, 9, 941, 10, 2, 316, 4, ...	1	5003
4	[736, 19, 37, 813, 45, 6723, 27, 626, 8, 2, 34...	1	5004
...	...	...	...
12995	[8, 15, 71, 12, 155, 6903, 3, 7, 2300, 352, 37...	0	17995
12996	[12, 155, 7, 420, 4, 228, 89, 206, 5157, 10, 5...	0	17996
12997	[216, 2, 379, 4, 7, 332, 179, 386, 160, 28, 11...	0	17997
12998	[7, 2787, 9, 1026, 7, 5376, 620, 1, 4303, 7, 1...	0	17998
12999	[2522, 988, 931, 3, 52, 206, 7143, 988, 931, 3...	0	17999
13000 rows × 3 columns

Missing values in Domain1 Training Data:
text     0
label    0
id       0
dtype: int64

Missing values in Domain2 Training Data:
text     0
label    0
id       0
dtype: int64

C:\Users\chang\AppData\Local\Temp\ipykernel_54868\741347689.py:8: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=domain1_class_counts.index, y=domain1_class_counts.values, palette='Blues_d')
C:\Users\chang\AppData\Local\Temp\ipykernel_54868\741347689.py:15: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=domain2_class_counts.index, y=domain2_class_counts.values, palette='Greens_d')

0    11500
1     1500
Name: label, dtype: int64
                                                    text  label     id
4938   [813, 4, 615, 14, 1134, 13, 340, 32, 532, 3644...      0   9938
7148   [28, 347, 2257, 5, 482, 6, 1912, 149, 3, 347, ...      0  12148
2704   [12, 216, 7, 2507, 817, 4, 6482, 3869, 57, 18,...      0   7704
12157  [12828, 4316, 603, 28, 2, 2475, 3030, 9, 2, 10...      0  17157
8791   [12, 437, 27, 1020, 1987, 4, 2, 217, 4, 2, 294...      0  13791
0    1500
1    1500
Name: label, dtype: int64
Shape of X_train_vec: (5120, 35061)
Shape of X_val_vec: (1280, 35061)
LSA-SVM Performance:
              precision    recall  f1-score   support

           0       0.86      0.60      0.71       653
           1       0.68      0.90      0.78       627

    accuracy                           0.75      1280
   macro avg       0.77      0.75      0.74      1280
weighted avg       0.77      0.75      0.74      1280

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best hyperparameters: {'lsa__n_components': 150, 'svm__C': 10}
Best LSA-SVM Performance:
              precision    recall  f1-score   support

           0       0.86      0.65      0.74       653
           1       0.71      0.89      0.79       627

    accuracy                           0.77      1280
   macro avg       0.78      0.77      0.76      1280
weighted avg       0.78      0.77      0.76      1280

SVM Performance:
              precision    recall  f1-score   support

           0       0.81      0.74      0.77       653
           1       0.75      0.82      0.78       627

    accuracy                           0.78      1280
   macro avg       0.78      0.78      0.78      1280
weighted avg       0.78      0.78      0.78      1280

Fitting 3 folds for each of 9 candidates, totalling 27 fits
Best Hyperparameters: {'learning_rate': 0.5, 'n_estimators': 200}
Validation Accuracy: 0.68828125
Fitting 3 folds for each of 4 candidates, totalling 12 fits
Best Hyperparameters: {'learning_rate': 0.5, 'n_estimators': 300}
Validation Accuracy: 0.6875
Fitting 3 folds for each of 9 candidates, totalling 27 fits
Best Hyperparameters: {'learning_rate': 1.0, 'n_estimators': 50}
Validation Accuracy: 0.6078125

Replicate Best Hyperparameters: {'learning_rate': 0.5, 'n_estimators': 200}